# Classification d'incidents avec un réseau  récurrent bidirectionnel et des *embeddings* Spacy

## 1. Création du jeu de données (*dataset*)

In [21]:
import json
import spacy
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [22]:
train_json_fn = "./data/incidents_train.json"
validation_json_fn = "./data/incidents_test.json"
test_json_fn = "./data/incidents_test.json"

In [23]:
#Fonction permettant de charger les données
def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
    return incident_list

In [24]:
train_list = load_incident_dataset(train_json_fn)
validation_list = load_incident_dataset(validation_json_fn)
test_list = load_incident_dataset(test_json_fn)

print("Nombre d'incidents dans train:", len(train_list))
print("Nombre d'incidents dans validation:", len(validation_list))
print("Nombre d'incidents dans test:", len(test_list))

Nombre d'incidents dans train: 2475
Nombre d'incidents dans validation: 531
Nombre d'incidents dans test: 531


In [25]:
#On divise nos listes en X(texte) et y(labels) pour chacun des sets

X_train = [instance["text"] for instance in train_list]
y_train = [instance["label"] for instance in train_list]

X_val = [instance["text"] for instance in validation_list]
y_val = [instance["label"] for instance in validation_list]

X_test = [instance["text"] for instance in test_list]
y_test = [instance["label"] for instance in test_list]

In [26]:
nb_classes = len(set(y_train))

In [27]:
nb_classes

9

## 2. Gestion de plongements de mots (*embeddings*)

In [28]:
#On telecharge nos embeddings/tokenizer spacy
spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 1.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [29]:
nlp = spacy.load('en_core_web_md')
embedding_size = nlp.meta['vectors']['width']

Ce bloc de code définit des tokens spéciaux pour le remplissage PAD et les mots inconnus UNK, associant à ces tokens des identifiants (0 et 1) et des vecteurs d'embedding de taille zéro pour chacun, et crée des dictionnaires pour mapper les identifiants aux mots (id2word) et les mots aux identifiants (word2id), ainsi que les identifiants aux embeddings correspondants (id2embedding).

In [30]:
padding_token = "<PAD>"   # mot 0
unk_token = "<UNK>"    # mot 1
zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = padding_token 
id2word[1] = unk_token 

word2id = {}
word2id[padding_token] = 0
word2id[unk_token] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding

In [31]:
word_index = 2  # Initialise le compteur d'index à 2, car 0 et 1 sont réservés pour <PAD> et <UNK>
vocab = word2id.keys()  # Récupère les mots déjà existants dans le dictionnaire word2id

# Parcourt toutes les incidents dans l'ensemble d'entraînement X_train
for incident in X_train:
    # Tokenise chaque question en mots à l'aide de spacy
    for word in nlp(incident):
        # Vérifie si le mot n'est pas déjà dans le vocabulaire
        if word.text not in vocab:
            # Si le mot est nouveau, lui attribue le prochain index disponible
            word2id[word.text] = word_index
            # Ajoute le mot et son index correspondant au dictionnaire id2word
            id2word[word_index] = word.text
            # Stocke le vecteur d'embedding du mot dans le dictionnaire id2embedding
            id2embedding[word_index] = word.vector
            # Incrémente l'index pour le prochain mot unique
            word_index += 1

La fonction ci dessous sera utilisée pour préparer un lot de données (batch) avant de le passer à notre réseau de neurones.

In [32]:
def pad_batch(batch):
    # Extrait les entrées (x) et les longueurs réelles (x_true_length) de chaque séquence dans le batch
    x = [x for x, y in batch]  # Récupère les séquences d'entrée
    x_true_length = [len(x) for x, y in batch]  # Calcule la longueur de chaque séquence d'entrée

    # Extrait et empile les cibles (y) de chaque élément du batch en un tensor
    y = torch.stack([y for x, y in batch], dim=0)

    # Rembourre les séquences d'entrée pour qu'elles aient toutes la même longueur et les retourne avec leurs longueurs réelles
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

Dans le bloc de code ci-dessous, une matrice d'embeddings est créée et remplie avec les vecteurs d'embeddings pour chaque mot du vocabulaire, en utilisant un dictionnaire qui mappe les identifiants de mots à leurs embeddings correspondants. Cette matrice est ensuite convertie en un tensor PyTorch, permettant son utilisation dans des réseaux de neurones, et la taille de la couche d'embeddings créée est affichée pour vérification.

In [33]:
# Calcule la taille du vocabulaire basé sur le dictionnaire id2embedding
vocab_size = len(id2embedding)
# Crée une matrice d'embeddings initialisée à zéro avec la taille du vocabulaire et la taille des embeddings
embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)

# Remplit la matrice d'embeddings avec les embeddings correspondants pour chaque ID de token
for token_id, embedding in id2embedding.items():
    embedding_layer[token_id, :] = embedding  # Affecte l'embedding à la ligne correspondant à l'ID du token

# Convertit la matrice d'embeddings de numpy à un tensor PyTorch
embedding_layer = torch.from_numpy(embedding_layer)

print("Taille de la couche d'embeddings:", embedding_layer.shape)


Taille de la couche d'embeddings: torch.Size([11642, 300])


## 3. Création de modèle(s)

Le réseau comprendra une couche d'embedding en entrée. Cette couche d'embedding sera utilisée pour générer les représentations vectorielles (embeddings) de chaque mot dans une phrase. Les inputs fournis à notre réseau seront des batchs contenant des listes de phrases, où chaque mot est encodé par son identifiant (ID) correspondant. Ces listes seront rembourrées (padded) pour assurer une longueur uniforme au sein d'un batch.

En plus des données de la phrase, nous fournirons également à notre fonction une variable x_length. Cette variable est importante pour la fonction pack_padded_sequence de PyTorch, qui permet de gérer efficacement les phrases de longueurs variables dans un batch. pack_padded_sequence crée une représentation compacte de ces batchs, en ignorant les éléments de padding, ce qui améliore l'efficacité du traitement par le réseau.

In [34]:
class LSTMClassifier(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes):
        super(LSTMClassifier, self).__init__()
        # Initialise une couche d'embedding à partir d'embeddings pré-entraînés
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        # Détermine la taille des embeddings
        self.embedding_size = embeddings.size()[1]   
        # Initialise un LSTM avec une seule couche, bidirectionnel
        self.rnn = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True, bidirectional=True)
        # Initialise une couche de classification qui prend en entrée la taille des états cachés * 2 (pour bidirectionnel) et produit le nombre de classes
        self.classification_layer = nn.Linear(hidden_state_size*2, nb_classes)
    
    def forward(self, x, x_lengths):
        # Passe les données d'entrée par la couche d'embedding
        x = self.embedding_layer(x)
        # Emballe les séquences rembourrées pour le traitement par LSTM
        packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        # Passe les données emballées par le LSTM et récupère l'état caché et l'état de cellule
        output, (h_n, c_n) = self.rnn(packed_batch)  # Utilise l'état caché de la dernière cellule
        # Concatène les états cachés des deux directions du LSTM bidirectionnel
        h_n_concatenated = torch.cat((h_n[0], h_n[1]), dim=1)
        # Passe l'état caché concaténé par la couche de classification
        x = self.classification_layer(h_n_concatenated)
        return x

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

Ici, nous définissons une classe Dataset qui permet de construire notre dataset à partir de listes de phrases et de leurs classes correspondantes, en utilisant le dictionnaire word2id défini précédemment. Ce dataset sera ensuite fourni à un DataLoader, qui se chargera de diviser le dataset en batches et d'effectuer le rembourrage nécessaire avec la fonction pad_batch, afin que toutes nos entrées aient la même taille.

In [35]:
class SpacyDataset(Dataset):
    def __init__(self, data , targets, word_to_id, spacy_model):
        self.data = data
        self.sequences = [None for _ in range(len(data))]
        self.targets = targets
        self.word2id = word_to_id
        self.tokenizer = spacy_model
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Vérifie si la séquence à l'index spécifié a déjà été tokenisée
        if self.sequences[index] is None:
            # Si non, tokenize la phrase à cet index et stocke le résultat dans self.sequences
            self.sequences[index] = self.tokenize(self.data[index]) 
        return LongTensor(self.sequences[index]), LongTensor([int(self.targets[index])]).squeeze(0)


    def tokenize(self, sentence):
        # Utilise le modèle spaCy pour tokeniser la phrase donnée
        tokens = [word.text for word in self.tokenizer(sentence)]
        # Convertit chaque token en son identifiant correspondant.
        # Si le token n'est pas trouvé dans dictionnaire word2id, utilise 1 par défaut, qui est l'identifiant pour <UNK> (mot inconnu).
        return [self.word2id.get(token, 1) for token in tokens]

## 5. Entraînement de modèle(s)

Pour l'entraînement du modèle, nous utiliserons la bibliothèque Poutyne, qui permet d'automatiser ce processus.

In [36]:
train_dataset = SpacyDataset(X_train, y_train, word2id, nlp)
valid_dataset = SpacyDataset(X_val, y_val, word2id, nlp)
test_dataset = SpacyDataset(X_test, y_test, word2id, nlp)


train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)

In [51]:
hidden_dimension = 100

In [52]:
directory_name = 'model_t2/_rnnBi'

model = LSTMClassifier(embedding_layer, hidden_dimension, nb_classes)
experiment = Experiment(directory_name, 
                                model, 
                                optimizer = "Adam", 
                                task="classification")
        
logging = experiment.train(train_dataloader, valid_dataloader, epochs=10, disable_tensorboard=True)

Epoch:  1/10 Train steps: 155 Val steps: 34 47.31s loss: 1.657397 acc: 41.616162 fscore_macro: 0.124085 val_loss: 1.555383 val_acc: 44.632768 val_fscore_macro: 0.168568
Epoch 1: val_acc improved from -inf to 44.63277, saving file to model_t2/_rnnBi/checkpoint_epoch_1.ckpt
Epoch:  2/10 Train steps: 155 Val steps: 34 46.78s loss: 1.291384 acc: 54.868687 fscore_macro: 0.257251 val_loss: 1.247357 val_acc: 55.743879 val_fscore_macro: 0.288286
Epoch 2: val_acc improved from 44.63277 to 55.74388, saving file to model_t2/_rnnBi/checkpoint_epoch_2.ckpt
Epoch:  3/10 Train steps: 155 Val steps: 34 48.18s loss: 1.000303 acc: 64.000000 fscore_macro: 0.369549 val_loss: 1.139469 val_acc: 61.016949 val_fscore_macro: 0.351226
Epoch 3: val_acc improved from 55.74388 to 61.01695, saving file to model_t2/_rnnBi/checkpoint_epoch_3.ckpt
Epoch:  4/10 Train steps: 155 Val steps: 34 48.44s loss: 0.836603 acc: 70.585859 fscore_macro: 0.480489 val_loss: 1.086270 val_acc: 64.030132 val_fscore_macro: 0.444195
Epoc

## 6. Évaluation et analyse de résultats

In [55]:
experiment.test(test_dataloader)

Found best checkpoint at epoch: 5
lr: 0.001, loss: 0.69636, acc: 75.1515, fscore_macro: 0.558061, val_loss: 1.07816, val_acc: 64.0301, val_fscore_macro: 0.442641
Loading checkpoint model_t2/_rnnBi/checkpoint_epoch_5.ckpt
Running test
Test steps: 34 1.10s test_loss: 1.078155 test_acc: 64.030132 test_fscore_macro: 0.442641          


{'time': 1.0988306250073947,
 'test_loss': 1.0781551772507572,
 'test_acc': 64.030131826742,
 'test_fscore_macro': 0.44264137744903564}

## Comparaison uni et bi ##

Notre RNN bidirectionnel a une exactitude de 64.03% et un score F macro de 0.4426, des performances inférieures à celles de votre RNN unidirectionnel, qui avait une exactitude de 67.42% et un score F macro de 0.587. Cela suggère que, pour votre ensemble de données spécifique, le RNN unidirectionnel est plus efficace, offrant une meilleure précision et un meilleur équilibre entre précision et rappel.

Bien que les RNN bidirectionnels soient souvent préférés pour leur capacité à intégrer des informations contextuelles étendues, il existe des scénarios spécifiques où un RNN unidirectionnel peut surpasser son homologue bidirectionnel en termes de performances, en particulier si les caractéristiques des données et les exigences de la tâche correspondent mieux à une analyse unidirectionnelle.

Le RNN unidirectionnel est meilleur que le bidirectionnel dans notre cas car il évite le surajustement. Les RNN bidirectionnels, bien que puissants, peuvent surajuster, surtout avec des ensembles de données limités ou moins divers. Le RNN unidirectionnel, en traitant les données dans une seule direction, peut mieux généraliser, offrant ainsi de meilleures performances sur des données non vu a l'entrainement.